## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [1]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.5
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset_normalizer, idna, urllib3
Required-by: anaconda-auth, anaconda-catalogs, anaconda-client, anaconda-project, conda, conda-build, conda-libmamba-solver, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, PyGithub, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [3]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.13.5
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve, typing-extensions
Required-by: conda-build, nbconvert


In [15]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup


In [ ]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}') # 실행 시점에 변수와 표현식을 평가하여 문자열을 동적으로 생성하는 방식
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.5
bs4 버전 = 4.13.5


### 1. 뉴스 제목 추출하기

In [ ]:
# IT/과학 뉴스 
req_param = {
    'sid': 105 # Session ID
}
# dict 타입으로 요청 파라미터 설정
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보 (사람처럼 보이게 하기 위함)
req_header = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}

# requests 의 get() 함수 호출하기 request: 페이지 소스 보기
res = requests.get(url, headers=req_header)
print(res.status_code)
print(res.ok)
print(type(res))
print(res)
#print(res.text)
# 응답(response)이 OK 이면
# 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성 
if res.ok: # beautifulsoup이 응답을 파싱하게
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")))
    #CSS 선택자를 사용해서 a tag 목록 가져오기
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
    print(type(a_tags), type(a_tags[0])) #[Tag, Tag, ....] 88개가 이런식으로 있음
    # <a> 태그 리스트 순회하기
    for a_tag in a_tags: #a_tag: tag object
        title = a_tag.text.strip()
        link = a_tag['href']
        print(title, link)
else:
    #응답(response)이 Error 이면 Status Code 출력
    print(f'Error Code = {res.status_code}')


https://news.naver.com/section/105
200
True
<class 'requests.models.Response'>
<Response [200]>
88
<class 'bs4.element.ResultSet'> <class 'bs4.element.Tag'>
사상 첫 연간 매출 12조원 돌파…네이버 최수연 “검색 넘어 에이전틱 AI로 수익성 강화” https://n.news.naver.com/mnews/article/016/0002596941
 https://n.news.naver.com/mnews/article/comment/016/0002596941
넷마블, 신작 흥행에 2025년 최대 매출 경신...영업익 3525억원 https://n.news.naver.com/mnews/article/014/0005474143
 https://n.news.naver.com/mnews/article/comment/014/0005474143
홍진경 "키 멈추는 방법은요"…'네이버 지식인 노출' 해프닝 https://n.news.naver.com/mnews/article/015/0005247160
 https://n.news.naver.com/mnews/article/comment/015/0005247160
"해킹 사고 반사익" 역대 최대 실적 쓴 LG U+, 매출 '15조' 고지 올랐다 https://n.news.naver.com/mnews/article/008/0005314490
 https://n.news.naver.com/mnews/article/comment/008/0005314490
구글, 고정밀 지도 국외 반출 보완서류 제출 https://n.news.naver.com/mnews/article/422/0000830908
 https://n.news.naver.com/mnews/article/comment/422/0000830908
구글 실적이 보여준 ‘AI의 힘’… 시장의 눈은 엔비디아로 https://n.news.naver.com/mnew

### 1.1 뉴스제목 추출하는 함수 선언하기

In [43]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section_name):  #print_new(103,'생활/문화') 
    sid = section_dict.get(section_name, '정치') # 값이 틀렸을 때 default 값이 정치
    print(sid)
    url = f'https://news.naver.com/section/{sid}'
    print(f'{section_name} 뉴스 {url}')
    req_header = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=req_header)
    if res.ok: # beautifulsoup이 응답을 파싱하게
        soup = BeautifulSoup(res.text, 'html.parser')   
        #CSS 선택자를 사용해서 a tag 목록 가져오기
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
        print(type(a_tags), type(a_tags[0])) #[Tag, Tag, ....] 88개가 이런식으로 있음
        # <a> 태그 리스트 순회하기
        for a_tag in a_tags: #a_tag: tag object
            title = a_tag.text.strip()
            link = a_tag['href']
            print(title, link)
    else:
        #응답(response)이 Error 이면 Status Code 출력
        print(f'Error Code = {res.status_code}')


In [44]:
print_news('경제')

101
경제 뉴스 https://news.naver.com/section/101
<class 'bs4.element.ResultSet'> <class 'bs4.element.Tag'>
'담합' 대한제분, 밀가루 가격 인하…제분업계 확산 여부 촉각 https://n.news.naver.com/mnews/article/421/0008755435
 https://n.news.naver.com/mnews/article/comment/421/0008755435
국제 금·은값, 다시 낙폭 확대…금 2%↓·은 12%↓ https://n.news.naver.com/mnews/article/001/0015888902
 https://n.news.naver.com/mnews/article/comment/001/0015888902
쿠팡 "개인정보 유출 추가…16만5천여개 계정" https://n.news.naver.com/mnews/article/422/0000830880
 https://n.news.naver.com/mnews/article/comment/422/0000830880
한화에어로-KAI 손잡고 '무인기 공동개발' 등 추진 https://n.news.naver.com/mnews/article/008/0005314994
 https://n.news.naver.com/mnews/article/comment/008/0005314994
창원 간 김정관 산업장관 “산학연 협력 통해 산업단지 AI 전환 이끌 것” https://n.news.naver.com/mnews/article/016/0002596962
 https://n.news.naver.com/mnews/article/comment/016/0002596962
李 대통령 자리 착각한 정의선에 '빵'터진 이재용 [영상] https://n.news.naver.com/mnews/article/018/0006214183
 https://n.news.naver.com/mnews/article/comment/018/00062141

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [ ]:
import requests
import os

req_header = {
    'referer':''
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg'
]

for img_url in img_urls:
    pass
    # requests 의 get(url, headers) 함수 호출하기 
        
        # binary 응답 데이터 가져오기
        
        # url에서 파일명만 추출하기
        
        # binday data를 file에 write하기


* 현재 요청된 페이지의 image 모두 다운로드 해보기

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.



200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청


In [ ]:
# 캡차 이미지 요청


In [ ]:
#  사용자 입력값 검증 요청



* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


### 5. 블로그 검색하기

In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
